## Refactor wavelet initialisation

Notebook to play around with classes and objects during the great Kymatio refactor of 2022

In [1]:
from sn_camels.models.models_factory import baseModelFactory, topModelFactory
from sn_camels.models.sn_hybrid_models import sn_HybridModel
from sn_camels.models.camels_models import get_architecture 
from sn_camels.camels.camels_dataset import *

/home/chrisp/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Check if CUDA available
if torch.cuda.is_available():
    print("CUDA Available")
    device = torch.device('cuda')
    use_cuda=True
else:
    print('CUDA Not Available')
    device = torch.device('cpu')
    use_cuda=False

CUDA Not Available


In [3]:
camels_path="/home/chrisp/Data/CAMELs/"
fparams    = camels_path+"/params_IllustrisTNG.txt"
fmaps      = [
              camels_path+"maps_Mtot.npy"
             ]

In [4]:
## dataloader options
seed=123

batch_size=10
splits=1
fmaps_norm=[None]
num_workers=1
rot_flip_in_mem=True
channels=1
features=1

train_loader = create_dataset_multifield('train', seed, fmaps, fparams, batch_size, splits, fmaps_norm,
                                                 num_workers=num_workers, rot_flip_in_mem=rot_flip_in_mem, verbose=True)

Found 1 channels
Reading data...
6.054e+09 < F(all|orig) < 2.176e+15
9.782 < F(all|resc)  < 15.338
-2.696 < F(all|norm) < 8.631
Channel 0 contains 7200 maps
-2.696 < F < 8.631



In [5]:
scatteringBase = baseModelFactory( #creat scattering base model
    architecture='scattering',
    J=2,
    N=256,
    M=256,
    channels=channels,
    max_order=2,
    initialization="Random",
    seed=234,
    learnable=True,
    lr_orientation=0.03,
    lr_scattering=0.03,
    skip=True,
    split_filters=True,
    filter_video=False,
    subsample=4,
    device=device,
    use_cuda=use_cuda,
    plot=False
)

## Now create a network to follow the scattering layers
## can be MLP, linear, or cnn at the moment
## (as in https://github.com/bentherien/ParametricScatteringNetworks/ )
top = topModelFactory( #create cnn, mlp, linear_layer, or other
    base=scatteringBase,
    architecture="linear_layer",
    num_classes=features,
    width=3,
    average=True,
    use_cuda=use_cuda
)

hybridModel = sn_HybridModel(scatteringBase=scatteringBase, top=top, use_cuda=use_cuda)
model=hybridModel
model.to(device=device)

(256, 256)
(128, 128)


/home/chrisp/.local/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/chrisp/.local/lib/python3.8/site-packages/sn_camels-0.0.1-py3.8.egg/sn_camels/scattering/create_filters.py:283: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2981.)


sn_HybridModel(
  (scatteringBase): sn_ScatteringBase()
  (top): sn_LinearLayer(
    (fc1): Linear(in_features=73, out_features=1, bias=True)
    (bn0): BatchNorm1d(73, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [6]:
x,y=next(iter(train_loader))
model.to(device=device)
x=x.to(device=device)

In [15]:
def do_convolutions(x, backend, J, phi, wavelets, max_order,
        split_filters, subsample):
    """ Function to take an input image and perform a series of scattering
    convolutions."""
    subsample_fourier = backend.subsample_fourier
    modulus = backend.modulus
    fft = backend.fft
    cdgmm = backend.cdgmm
    concatenate = backend.concatenate
    
    wavelets = wavelets.real.contiguous().unsqueeze(3)

    # Define lists for output.
    out_S_0, out_S_1, out_S_2 = [], [], []
    
    ## Map to complex
    complex_maps = x.new_zeros(x.shape + (2,))
    complex_maps[..., 0] = x

    U_0_c = fft(complex_maps, 'C2C')
    print(U_0_c.shape)
    

    # First low pass filter
    U_1_c = cdgmm(U_0_c, phi[0])
    U_1_c = subsample_fourier(U_1_c, k=subsample)


    S_0 = fft(U_1_c, 'C2R', inverse=True)

    out_S_0.append({'coef': S_0})

    if split_filters:
        for n1 in range(int(len(wavelets)/2)):

            ## Wavelet convolution
            
            U_1_c = cdgmm(U_0_c, wavelets[n1])

            U_1_c = fft(U_1_c, 'C2C', inverse=True)
            U_1_c = modulus(U_1_c)
            U_1_c = fft(U_1_c, 'C2C')

            ## Second low pass filter
            S_1_c = cdgmm(U_1_c, phi[0])
            S_1_c = subsample_fourier(S_1_c, k=subsample)

            S_1_r = fft(S_1_c, 'C2R', inverse=True)

            out_S_1.append({'coef': S_1_r})

            if max_order < 2:
                continue
            for n2 in range(int(len(psi)/2),len(psi)):
                

                U_2_c = cdgmm(U_1_c, wavelets[n2])
                U_2_c = fft(U_2_c, 'C2C', inverse=True)
                U_2_c = modulus(U_2_c)
                U_2_c = fft(U_2_c, 'C2C')

                ## Low pass filter
                S_2_c = cdgmm(U_2_c, phi[0])
                
                S_2_c = subsample_fourier(S_2_c, k=subsample)

                S_2_r = fft(S_2_c, 'C2R', inverse=True)
                

                out_S_2.append({'coef': S_2_r})
    else:
        for n1 in range(len(wavelets)):
            ## Wavelet convolution
            U_1_c = cdgmm(U_0_c, wavelets[n1])

            U_1_c = fft(U_1_c, 'C2C', inverse=True)
            U_1_c = modulus(U_1_c)
            U_1_c = fft(U_1_c, 'C2C')

            ## Second low pass filter
            S_1_c = cdgmm(U_1_c, phi[0])
            S_1_c = subsample_fourier(S_1_c, k=subsample)

            S_1_r = fft(S_1_c, 'C2R', inverse=True)

            out_S_1.append({'coef': S_1_r})

            if max_order < 2:
                continue
            for n2 in range(len(wavelets)):
                
                U_2_c = cdgmm(U_1_c, wavelets[n2])
                U_2_c = fft(U_2_c, 'C2C', inverse=True)
                U_2_c = modulus(U_2_c)
                U_2_c = fft(U_2_c, 'C2C')

                ## Low pass filter
                S_2_c = cdgmm(U_2_c, phi[0])
                S_2_c = subsample_fourier(S_2_c, k=subsample)
                S_2_r = fft(S_2_c, 'C2R', inverse=True)
                

                out_S_2.append({'coef': S_2_r})

    out_S = []
    out_S.extend(out_S_0)
    out_S.extend(out_S_1)
    out_S.extend(out_S_2)

    out_S = concatenate([x['coef'] for x in out_S])

    return out_S

def convolve_fields(input, backend, J, phi, wavelets, max_order, split_filters, subsample):
    """  
        Wrapper function for a loop that will convovle each wavelet with the input fields

        Parameters:
            input      -- input data
            psi        -- dictionnary of filters that is used in the kymatio code
            split_filters -- split first and second order filters
        Returns:
            S -- Fields after being convolved with wavelets
    """

    batch_shape = input.shape[:-2]
    signal_shape = input.shape[-2:]

    input = input.reshape((-1,) + signal_shape)

    S = do_convolutions(input, backend, J, phi, wavelets,
                        max_order, split_filters, subsample)

    ## S will always be a numpy array
    scattering_shape = S.shape[-3:]
    S = S.reshape(batch_shape + scattering_shape)

    return S

In [16]:
x.shape

torch.Size([10, 1, 256, 256])

In [17]:
convolve_fields(x, scatteringBase.backend, scatteringBase.J, scatteringBase.phi, scatteringBase.wavelets,
                                    scatteringBase.max_order, scatteringBase.split_filters,scatteringBase.subsample)

torch.Size([10, 256, 256, 2])


TypeError: The input should be complex (i.e. last dimension is 2).